# Create fine-tuning data

In [1]:
import pandas as pd
import json 

In [2]:
ls

Codebook for extractive summary.docx
CreateFineTuningData.ipynb
InterAnnotator2.ipynb
SynopsisFinder.py
T. Underwood Fund Balance Summary_08312025_nr.xlsx
all_synopses.csv
annotations/
annotations2/
avery_synopses.xlsx
lab_annotated.csv
longer_synopses.csv
longer_synopses.json
new_gutenberg_synopses.json
no_synopses_sample.csv
paragraphs-extracted.json
selected_long_synopses.csv
selected_longer2_synopses.csv
selected_longer_synopses.csv
selected_new_gutenberg_synopses.csv
selected_synopses.csv
shared_longer_synopses.csv
shared_synopses.csv
some samples.xlsx
synopses.json
synopses_for_annotation.csv
synopses_output_gemini.json
synopsis_unpacker.ipynb
test_cases_for_codebook.csv
test_cases_for_codebook_ted.csv
twelve_test_cases.csv


In [3]:
qiyan_data = json.load(open('synopses_output_gemini.json'))

In [5]:
qiyan_data[0]

{'position': 61,
 'filename': '70203.txt',
 'passage': 'Schubert’s fame rests chiefly on his songs, but in the domain of\ninstrumental music he also did some remarkable work, although it\ncan hardly be said that he fundamentally influenced the art in\nthis direction. He lived (1797-1828) in Vienna at the same time as\nBeethoven, and many of his compositions show distinctly how much\nhe was influenced by the genius of that great musician. Schubert’s\nworks are distinguished by a spontaneous flow of beautiful melody.\nHe possessed, by right of nature, the utmost artistic opulence, but\nunfortunately, owing chiefly to his birth and early circumstances, this\nwas not consolidated by that profounder culture which is necessary for\nthe fullest expression of human capacity. Not that Schubert failed\nto reach a most exalted position, for works like the B♭ Piano Trio,\nthe C major Quintett, that noble fragment the B minor Symphony, and\nespecially his best songs, proclaim him unsurpassed by any

In [6]:
print(qiyan_data[0]['passage'])

Schubert’s fame rests chiefly on his songs, but in the domain of
instrumental music he also did some remarkable work, although it
can hardly be said that he fundamentally influenced the art in
this direction. He lived (1797-1828) in Vienna at the same time as
Beethoven, and many of his compositions show distinctly how much
he was influenced by the genius of that great musician. Schubert’s
works are distinguished by a spontaneous flow of beautiful melody.
He possessed, by right of nature, the utmost artistic opulence, but
unfortunately, owing chiefly to his birth and early circumstances, this
was not consolidated by that profounder culture which is necessary for
the fullest expression of human capacity. Not that Schubert failed
to reach a most exalted position, for works like the B♭ Piano Trio,
the C major Quintett, that noble fragment the B minor Symphony, and
especially his best songs, proclaim him unsurpassed by any musician
of modern times; but out of the very profusion of his ideas

In [7]:
# Function that accepts a string of English text and splits it into sentences
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def split_into_sentences(text):
    return sent_tokenize(text)

[nltk_data] Downloading package punkt to /Users/tunder/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Go through each element of qiyan_data. For each element, we're going
# to add the model_synopsis_sentence field to the passage field. But
# the position of the added sentence will vary randomly:
#
# 30% cases: beginning
# 40% cases: middle, between two random sentences
# 30% cases: end
import random
for entry in qiyan_data:
    passage = entry['passage']
    model_synopsis_sentence = entry['model_synopsis_sentence']
    sentences = split_into_sentences(passage)
    rand_val = random.random()
    if rand_val < 0.3:
        # beginning
        new_passage = model_synopsis_sentence + " " + passage
    elif rand_val < 0.7:
        # middle
        if len(sentences) > 1:
            insert_pos = random.randint(1, len(sentences)-1)
            new_passage = " ".join(sentences[:insert_pos]) + " " + model_synopsis_sentence + " " + " ".join(sentences[insert_pos:])
        else:
            new_passage = passage + " " + model_synopsis_sentence
    else:
        # end
        new_passage = passage + " " + model_synopsis_sentence
    entry['passage_with_synopsis'] = new_passage
